In [ ]:
# export
from fastai.datasets import URLs, untar_data
from pathlib import Path
import torch, re, PIL, os, mimetypes, csv, operator, pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from typing import *
import pandas as pd, numpy as np
from enum import Enum
from torch import tensor,Tensor

## Data block API from config class

### Core helper functions

In [ ]:
# export
def test(a,b,cmp,cname=None,tst_name=''):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{tst_name},{cname}:\n{a}\n{b}"

def test_eq(a,b,tst_name=''): 
    if isinstance(a, np.ndarray) or (isinstance(a, Tensor) and not len(a.shape) == 0):
        assert len(a) == len(b), f"{tst_name}, lengths mismatch:\n{a}\n{b}"
        test(a,b,lambda x,y: (x == y).all(),'==',tst_name)
    else: test(a,b,operator.eq,'==',tst_name)

In [ ]:
# export
def noop(x): return x

In [ ]:
# test
test_eq(noop(1),1)

In [ ]:
# export
def listify(o):
    "Make `o` a list."
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
# test
test_eq(listify(None), [])
test_eq(listify([1,2,3]), [1,2,3])
test_eq(listify('abc'), ['abc'])
test_eq(listify(range(0,3)), [0,1,2])

In [ ]:
# export
def compose(x, funcs, *args, order_key='_order', **kwargs):
    "Apply all `funcs` to `x` in order, pass along `args` and `kwargs`."
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, *args, **kwargs)
    return x

In [ ]:
# test
# basic behavior
def _test_f1(x, a=2): return x**a
def _test_f2(x, a=2): return a*x
test_eq(compose(2, [_test_f1, _test_f2]), 8)
# order
_test_f1._order = 1
test_eq(compose(2, [_test_f1, _test_f2]), 16)
#args
test_eq(compose(2, [_test_f1, _test_f2], 3), 216)
#kwargs
test_eq(compose(2, [_test_f1, _test_f2], a=3), 216)

In [ ]:
# export
def uniqueify(x, sort=False):
    "Return the unqiue elements in `x`, optionally `sort`-ed."
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

In [ ]:
# test
test_eq(set(uniqueify([1,1,0,5,0,3])), {0,1,3,5})
test_eq(uniqueify([1,1,0,5,0,3], sort=True), [0,1,3,5])

In [ ]:
# export
def setify(o): return o if isinstance(o,set) else set(listify(o))

In [ ]:
# test
test_eq(setify(None), set())
test_eq(setify('abc'), {'abc'})
test_eq(setify([1,2,2]), {1,2})
test_eq(setify(range(0,3)), {0,1,2})
test_eq(setify({1,2}), {1,2})

In [ ]:
# export
def onehot(x, c):
    "Return the one-hot encoded tensor for `x` with `c` classes."
    res = torch.zeros(c)
    res[x] = 1.
    return res

In [ ]:
# test
test_eq(onehot(1,5), tensor([0.,1.,0.,0.,0.]))
test_eq(onehot([1,3],5), tensor([0.,1.,0.,1.,0.]))
test_eq(onehot(tensor([1,3]),5), tensor([0.,1.,0.,1.,0.]))
test_eq(onehot([True,False,True,True,False],5), tensor([1.,0.,1.,1.,0.]))
test_eq(onehot([],5), tensor([0.,0.,0.,0.,0.]))

In [ ]:
# export
def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=False, include=None):
    "Get all the files in `path` with optional `extensions`."
    path = Path(path)
    extensions = setify(extensions)
    extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
            if include is not None and i==0: d[:] = [o for o in d if o in include]
            else:                            d[:] = [o for o in d if not o.startswith('.')]
            res += _get_files(p, f, extensions)
        return res
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        return _get_files(path, f, extensions)

In [ ]:
# test
path = untar_data(URLs.MNIST_TINY)
test_eq(len(get_files(path/'train'/'3')), 346)
test_eq(len(get_files(path/'train'/'3', extensions='.png')), 346)
test_eq(len(get_files(path/'train'/'3', extensions='.jpg')), 0)
test_eq(len(get_files(path/'train', extensions='.png')), 0)
test_eq(len(get_files(path/'train', extensions='.png', recurse=True)), 709)
test_eq(len(get_files(path, extensions='.png', recurse=True, include=['train'])), 709)
test_eq(len(get_files(path, extensions='.png', recurse=True, include=['train', 'test'])), 729)

In [ ]:
# export
def grab_idx(batch, i):
    "Return the `i`-th sample in `batch`"
    return [grab_idx(b,i) for b in batch] if isinstance(batch, (list,tuple)) else batch[i].detach().cpu()

In [ ]:
# test
test_eq(grab_idx(tensor([1,2]), 1), 2)
test_eq(grab_idx([tensor([1,2]), tensor([3,4])], 1), [2,4])

In [ ]:
# export
def read_column(df, col_name, prefix='', suffix='', delim=None):
    "Read `col_name` in `df`, optionnally adding `prefix` or `suffix`."
    values = df[col_name].values.astype(str)
    values = np.char.add(np.char.add(prefix, values), suffix)
    if delim is not None:
        values = np.array(list(csv.reader(values, delimiter=delim)))
    return values

In [ ]:
# test
df = pd.DataFrame({'a': ['cat', 'dog', 'car'], 'b': ['a b', 'c d', 'a e']})
test_eq(read_column(df, 'a'), np.array(['cat', 'dog', 'car']))
test_eq(read_column(df, 'a', prefix='o'), np.array(['ocat', 'odog', 'ocar']))
test_eq(read_column(df, 'a', suffix='.png'), np.array(['cat.png', 'dog.png', 'car.png']))
test_eq(read_column(df, 'b', delim=' '), np.array([['a','b'], ['c','d'], ['a','e']]))

### Transform

In [ ]:
# export
class Transform():
    "A basic class to transform some data."
    _order=0
    def __call__(self, o):  return o
    def undo(self, o):      return o
    def setup(self, items): pass

NB: undo is for the transforms you want to undo when showing data. You want to undo one-hot encoding but not opening an imare or data augmentation transforms for instance.

In [ ]:
# export
class Item():
    "A basic class for representing some data type."
    default_tfms = None
    default_tfms_xy = None
    tfm_kwargs = None
    
    def init_tfms(self, xy=False):
        kwargs = self.tfm_kwargs if self.tfm_kwargs is not None else {}
        return [t(**kwargs) for t in listify(self.default_tfms_xy if xy else self.default_tfms)]
    
    def show(self, x, ax):
        "How to show one element `x` on `ax`."
        raise NotImplementedError

### Data block API core

One ItemList to contain them all

In [ ]:
# export
class ItemList():
    def __init__(self, items, tfms=None, item_type=Item()): 
        self.items,self.tfms,self.item_type = items,tfms,item_type
        for tfm in listify(self.tfms): tfm.setup(self.items)
    def _get(self, i): return compose(i, self.tfms)
    def __getitem__(self, idx):
        try: return self._get(self.items[idx])
        except TypeError:
            if isinstance(idx[0],bool):
                assert len(idx)==len(self) # bool mask
                return [self._get(o) for m,o in zip(idx,self.items) if m]
            return [self._get(self.items[i]) for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    
    def deproc(self, o): 
        "Reverse transforms on `o`."
        return compose(o, [t.undo for t in reversed(listify(self.tfms))])
    
    def show(self, i, ax=None):
        "Show item `i` on `ax`."
        x = self.deproc(self[i])
        if ax is None: _,ax = plt.subplots(1,1)
        self.item_type.show(x, ax)

In [ ]:
# export
class LabeledData():
    def __init__(self, x, y, tfms=None): 
        self.x,self.y,self.tfms = x,y,tfms
        for tfm in listify(self.tfms): tfm.setup(self)
    def __repr__(self):        return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx): return compose((self.x[idx],self.y[idx]), self.tfms)
    def __len__(self):         return len(self.x)
    
    def deproc(self, o):
        "Reverse transforms on `o`."
        (x,y) = compose(o, [t.undo for t in reversed(listify(self.tfms))])
        return (self.x.deproc(x),self.y.deproc(y))
    
    def show(self, i, ax=None):
        "Show item and label `i` on `ax`."
        (x,y) = self.deproc(self[i])
        if ax is None: _,ax = plt.subplots(1,1)
        self.x.item_type.show(x, ax)
        self.y.item_type.show(y, ax)

Dataloader and DataBunch

In [ ]:
# export
from torch.utils.data.dataloader import DataLoader
def get_dl(ds, bs, shuffle=False, drop_last=False, **kwargs):
    "Basic function to get a `DataLoader`"
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, drop_last=drop_last, **kwargs)

In [ ]:
# export
class DataBunch():
    "Basic wrapper around several `DataLoader`."
    def __init__(self, train_dl, valid_dl):
        self.train_dl,self.valid_dl = train_dl,valid_dl
        
    @property
    def train_ds(self): return self.train_dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dataset

Main class to represent any kind of data. User provides the `get_x_cls` and `get_y_cls` then the four functions. At init everything is constructed with optionals transforms or custom instances of `get_x`/`get_y`.

In [ ]:
# export
class DataBlock():
    "Main class to represent a dataset. Subclass the 2 properties and 4 methods below to your need."
    x_cls = Item #Type of input
    y_cls = Item #Type of targer
    def get_source(self):         
        "Return the source of your data (path, dataframe...), optionally download it."
        raise NotImplementedError
    def get_items(self, source):  
        "Use `source` to return the list of all items."
        raise NotImplementedError
    def split(self, items):       
        "Explain how so split the `items`. Return two disjoint lists of indices or two boolean masks."
        raise NotImplementedError
    def label(self, items):       
        "Explain how to label your `items`. Return a list of labels."
        raise NotImplementedError
        
    def __init__(self, tfms=None, tfms_x=None, tfms_y=None, tfms_xy=None):
        self.source = self.get_source()
        items = ItemList(self.get_items(self.source)) #Just for fancy indexing
        split_idx = self.split(items)
        labels = ItemList(self.label(items))          #Just for fancy indexing
        if tfms_x is None: tfms_x = self.x_cls().init_tfms()
        if tfms_y is None: tfms_y = self.y_cls().init_tfms()
        x_train,x_valid = map(lambda o: ItemList(items[o],  tfms=tfms_x, item_type=self.x_cls()), split_idx)
        y_train,y_valid = map(lambda o: ItemList(labels[o], tfms=tfms_y, item_type=self.y_cls()), split_idx)
        if tfms_xy is None: 
            tfms_xy = self.x_cls().init_tfms(xy=True) + self.y_cls().init_tfms(xy=True)
        self.train = LabeledData(x_train, y_train, tfms=listify(tfms) + tfms_xy)
        self.valid = LabeledData(x_valid, y_valid, tfms=listify(tfms) + tfms_xy)
    
    def databunch(self, bs=64, **kwargs):
        "How to convert to a `DataBunch`. Subclass if needed."
        dls = [get_dl(ds, bs, shuffle=s, drop_last=s, **kwargs) for (ds, s) in zip([self.train, self.valid], [True,False])]
        return DataBunch(*dls)

### First Items

How to get images

In [ ]:
class ImageOpener(Transform):
    def __init__(self, **kwargs): super().__init__() #This line can be removed when we make this a ImageTransform
    def __call__(self, o): return PIL.Image.open(o)

In [ ]:
# export
class Image(Item):
    default_tfms = ImageOpener
    def __init__(self, cmap=None, alpha=1.): self.cmap,self.alpha = cmap,alpha
    
    def show(self, x, ax):
        ax.imshow(x, cmap=self.cmap, alpha=self.alpha)
        ax.axis('off')

How to get categories

In [ ]:
class CategoryEncoder(Transform):
    "Encodes a categorical variable to index."
    def __init__(self): self.vocab=None
    
    def setup(self, items):
        if self.vocab is not None: return
        self.vocab = uniqueify(items, sort=True)
        self.otoi  = {v:k for k,v in enumerate(self.vocab)}
    
    def __call__(self, o): return self.otoi[o]
    def undo(self, i):    return self.vocab[i]

In [ ]:
# export
class Category(Item):
    default_tfms = CategoryEncoder
    def show(self, x, ax): ax.set_title(x)

In [ ]:
tfm = CategoryEncoder()
#Even if 'dog' is the first class, vocab is sorted for reproducibility
tfm.setup(['dog', 'cat', 'cat', 'dog', 'cat', 'dog'])
test_eq(tfm.vocab,['cat', 'dog'])
test_eq([tfm(o) for o in ['dog', 'cat', 'cat']], [1,0,0])
test_eq(tfm('cat'),0)
test_eq(tfm.undo(1),'dog')

### Helper functions

To get image files

In [ ]:
# export
image_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))

In [ ]:
# export
def get_image_files(path, include=None):
    "Get image files in `path` recursively."
    return get_files(path, extensions=image_extensions, recurse=True, include=include)

In [ ]:
# test
path = untar_data(URLs.MNIST_TINY)
test_eq(len(get_image_files(path)),1428)
test_eq(len(get_image_files(path/'train')),709)
test_eq(len(get_image_files(path, include='train')),709)
test_eq(len(get_image_files(path, include=['train','valid'])),1408)

To split

In [ ]:
# export
def random_splitter(items, valid_pct=0.2, seed=None):
    "Split `items` between train/val with `valid_pct` randomly."
    if seed is not None: torch.manual_seed(seed)
    rand_idx = torch.randperm(len(items))
    cut = int(valid_pct * len(items))
    return rand_idx[cut:],rand_idx[:cut]

In [ ]:
#test
trn,val = random_splitter([0,1,2,3,4,5], seed=42)
test_eq(trn, tensor([3, 2, 4, 1, 5]))
test_eq(val, tensor([0]))

In [ ]:
# export
def _grandparent_mask(items, name):
    return [(o.parent.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-2]) == name for o in items]

def grandparent_splitter(items, train_name='train', valid_name='valid'):
    "Split `items` from the grand parent folder names (`train_name` and `valid_name`)."
    return _grandparent_mask(items, train_name),_grandparent_mask(items, valid_name)

In [ ]:
path = untar_data(URLs.MNIST_TINY)

In [ ]:
#test
#With string filenames
path = untar_data(URLs.MNIST_TINY)
items = [path/'train'/'3'/'9932.png', path/'valid'/'7'/'7189.png', 
         path/'valid'/'7'/'7320.png', path/'train'/'7'/'9833.png',  
         path/'train'/'3'/'7666.png', path/'valid'/'3'/'925.png',
         path/'train'/'7'/'724.png', path/'valid'/'3'/'93055.png']
trn,val = grandparent_splitter(items)
test_eq(trn,[True,False,False,True,True,False,True,False])
test_eq(val,[False,True,True,False,False,True,False,True])

To label

In [ ]:
# export
def parent_labeller(items):
    "Label `items` with the parent folder name."
    return [o.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-1] for o in items]

In [ ]:
# test
test_eq(parent_labeller(items),['3','7','7','7','3','3','7','3'])

In [ ]:
# export
def func_labeller(items, func):
    "Label `items` according to `func`."
    return [func(o) for o in items]

In [ ]:
# test
test_eq(func_labeller(items, lambda x: int(x.parent.name)+1),[4,8,8,8,4,4,8,4])

In [ ]:
# export
def re_labeller(items, pat):
    "Label `items` with a regex `pat`."
    pat = re.compile(pat)
    def _inner(o):
        res = pat.search(str(o))
        assert res,f'Failed to find "{pat}" in "{o}"'
        return res.group(1)
    return func_labeller(items, _inner)

In [ ]:
# test
pat = re.compile(r'/([^/]+)/\d+.png$')
test_eq(re_labeller(items, pat),['3','7','7','7','3','3','7','3'])

### First data:

Integration test

In [ ]:
class PetsData(DataBlock):
    x_cls = Image
    y_cls = Category
    
    def get_source(self):        return untar_data(URLs.PETS)
    def get_items(self, source): return get_image_files(source/"images")
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return re_labeller(items, pat = r'/([^/]+)_\d+.jpg$')

In [ ]:
data = PetsData()

In [ ]:
data.train.show(0)

## Transforms

In [ ]:
# export
TfmY = Enum('TfmY', 'No Mask Image Point Bbox')

In [ ]:
# export
class ImageTransform(Transform): 
    "Basic class for data augmentation transforms."
    _order=0
    _tfm_y_func={TfmY.Image: 'apply_img',   TfmY.Mask: 'apply_mask', TfmY.No: 'noop',
                 TfmY.Point: 'apply_point', TfmY.Bbox: 'apply_bbox'}
    _undo_y_func={TfmY.Image: 'unapply_img',   TfmY.Mask: 'unapply_mask', TfmY.No: 'noop',
                  TfmY.Point: 'unapply_point', TfmY.Bbox: 'unapply_bbox'}
    
    def __init__(self, tfm_y=TfmY.No): self.tfm_y=tfm_y
        
    def apply(self, x):       return x
    def apply_img(self, y):   return self.apply(y)
    def apply_mask(self, y):  return self.apply_img(y)
    def apply_point(self, y): return y
    def apply_bbox(self, y):  return self.apply_point(y)
    
    def randomize(self): pass
    
    def __call__(self, o):
        (x,y) = o
        self.x = x #Saves the x in case it's needed in the apply for y (x.size for apply_point for instance)
        self.randomize() #Ensures we have the same state for x and y
        return self.apply(x),getattr(self, self._tfm_y_func[self.tfm_y], noop)(y)
    
    def unapply(self, x):       return x
    def unapply_img(self, y):   return self.unapply(y)
    def unapply_mask(self, y):  return self.unapply_img(y)
    def unapply_bbox(self, y):  return self.unapply_point(y)
    
    def undo(self, o):
        (x,y) = o
        return self.unapply(x),getattr(self, self._undo_y_func[self.tfm_y], noop)(y)

In [ ]:
#export
class DecodeImg(ImageTransform):
    "Convert regular image to RGB, masks to L mode."
    def __init__(self, tfm_y=TfmY.No, mode_x='RGB', mode_y=None):
        super().__init__(tfm_y)
        self.mode_x,self.mode_y = mode_x,mode_y
        
    def apply(self, x):       return x.convert(self.mode_x)
    def apply_image(self, y): return y.convert(self.mode_x if self.mode_y is None else self.mode_y)
    def apply_mask(self, y):  return y.convert('L' if self.mode_y is None else self.mode_y)

In [ ]:
#export
class ResizeFixed(ImageTransform):
    "Resize image to `size` using `mode_x` (and `mode_y` on targets)."
    _order=10
    def __init__(self, size, tfm_y=TfmY.No, mode_x=PIL.Image.BILINEAR, mode_y=None):
        super().__init__(tfm_y)
        if isinstance(size,int): size=(size,size)
        size = (size[1],size[0]) #PIL takes size in the otherway round
        self.size,self.mode_x,self.mode_y = size,mode_x,mode_y
        
    def apply(self, x):       return x.resize(self.size, self.mode_x)
    def apply_image(self, y): return y.resize(self.size, self.mode_x if self.mode_y is None else self.mode_y)
    def apply_mask(self, y):  return y.resize(self.size, PIL.Image.NEAREST if self.mode_y is None else self.mode_y)

In [ ]:
#export
class ToByteTensor(ImageTransform):
    "Transform our items to byte tensors."
    _order=20
    
    def apply(self, x):
        res = torch.ByteTensor(torch.ByteStorage.from_buffer(x.tobytes()))
        w,h = x.size
        return res.view(h,w,-1).permute(2,0,1)
    
    def unapply(self, x): return x[0] if x.shape[0] == 1 else x.permute(1,2,0)

In [ ]:
#export
class ToFloatTensor(ImageTransform):
    "Transform our items to float tensors (int in the case of mask)."
    _order=20
    def __init__(self, tfm_y=TfmY.No, div_x=255., div_y=None):
        super().__init__(tfm_y)
        self.div_x,self.div_y = div_x,div_y
    def apply(self, x):      return x.float().div_(self.div_x)
    def apply_mask(self, x): 
        return x.long() if self.div_y is None else x.long().div_(self.div_y)

In [ ]:
tfms = [DecodeImg(), ResizeFixed(128), ToByteTensor(), ToFloatTensor()]

Integration test

In [ ]:
data = PetsData(tfms=tfms)

In [ ]:
data.train.show(1)

## Try different data

### MNIST

In [ ]:
class MnistData(DataBlock):
    x_cls = Image
    y_cls = Category
    
    def get_source(self):        return untar_data(URLs.MNIST)
    def get_items(self, source): return get_image_files(source)
    def split(self, items):      return grandparent_splitter(items, train_name='training', valid_name='testing')
    def label(self, items):      return parent_labeller(items)

In [ ]:
data = MnistData(tfms=[ToByteTensor(), ToFloatTensor()])

In [ ]:
data.train.show(1)

cmap is specified in the `item_get` for inputs.

In [ ]:
data.train.x.item_type.cmap='gray'

In [ ]:
data.train.show(1)

### Planet

In [ ]:
path = untar_data(URLs.PLANET_SAMPLE)

In [ ]:
path.ls()

In [ ]:
df = pd.read_csv(path/'labels.csv')

In [ ]:
df.head()

In [ ]:
class MultiCategoryEncoder(Transform):
    "Encodes a categorical variable to index."
    def __init__(self, do_encode=True, classes=None): 
        assert do_encode or classes is not None, "If you use one_hot encoded items, please provide classes."
        self.vocab,self.do_encode=None,do_encode
        self.vocab = classes
    
    def setup(self, items):
        if self.vocab is not None: return
        vocab = set()
        for c in items: vocab = vocab.union(set(c))
        self.vocab = list(vocab)
        self.vocab.sort()
        self.otoi  = {v:k for k,v in enumerate(self.vocab)}
    
    def __call__(self, item): 
        if not self.do_encode: return item
        return onehot([self.otoi[o] for o in item if o in self.otoi], len(self.vocab))
    
    def undo(self, o): return [self.vocab[i] for i,v in enumerate(o) if v==1.]

In [ ]:
# export
class MultiCategory(Item):
    default_tfms = MultiCategoryEncoder
    def show(self, x, ax): ax.set_title(';'.join(x))

In [ ]:
# test
tfm = MultiCategoryEncoder()
#Even if 'c' is the first class, vocab is sorted for reproducibility
tfm.setup([['c','a'], ['a','b'], ['b']])
test_eq(tfm.vocab,['a','b','c'])

test_eq(tfm(['b','a']),tensor([1.,1.,0.]))
test_eq(tfm.undo(tensor([1.,0.,1.])),['a','c'])

In [ ]:
class PlanetData(DataBlock):
    x_cls = Image
    y_cls = MultiCategory
    
    def get_source(self):        
        self.path = untar_data(URLs.PLANET_SAMPLE)
        return pd.read_csv(path/'labels.csv')
    def get_items(self, source): return read_column(source, 'image_name', prefix=f'{self.path}/train/', suffix='.jpg')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return read_column(self.source, 'tags', delim=' ')

In [ ]:
data = PlanetData(tfms=tfms)

In [ ]:
data.train.show(0)

In [ ]:
classes = data.train.y.tfms[0].vocab
otoi = {s:i for i,s in enumerate(classes)}

In [ ]:
class PlanetData1(DataBlock):
    x_cls = Image
    y_cls = MultiCategory
    
    def get_source(self):        
        self.path = untar_data(URLs.PLANET_SAMPLE)
        return pd.read_csv(path/'labels.csv')
    def get_items(self, source): return read_column(source, 'image_name', prefix=f'{self.path}/train/', suffix='.jpg')
    def split(self, items):      return random_splitter(items)
    def label(self, items):  
        #This is just for the sake of using one-hot encoded labels, but imagine we have a dataset where it's the case.
        tags = read_column(self.source, 'tags', delim=' ')
        labels = []
        for t in tags:
            x = torch.zeros(len(classes))
            idx = [otoi.get(l,None) for l in t]
            idx = [i for i in idx if i is not None]
            x[idx] = 1.
            labels.append(x)
        return labels

In [ ]:
data = PlanetData1(tfms=tfms, tfms_y=MultiCategoryEncoder(do_encode=False, classes=classes))

In [ ]:
data.train.show(0)

### Camvid

In [ ]:
# export
class SegmentMask(Image):
    "An `ItemGetter` for segmentation mask targets."
    tfm_kwargs = {'tfm_y': TfmY.Mask}
    def __init__(self, cmap='tab20', alpha=0.5): 
        super().__init__(cmap=cmap, alpha=alpha)

In [ ]:
class CamvidData(DataBlock):
    x_cls = Image
    y_cls = SegmentMask
    
    def get_source(self):        return untar_data(URLs.CAMVID_TINY)      
    def get_items(self, source): return get_image_files(source/'images')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        path_lbl = self.source/'labels'
        codes = np.loadtxt(self.source/'codes.txt', dtype=str)
        return func_labeller(items, lambda x: path_lbl/f'{x.stem}_P{x.suffix}')

In [ ]:
tfms = [DecodeImg(tfm_y=TfmY.Mask), ResizeFixed(128, tfm_y=TfmY.Mask), ToByteTensor(tfm_y=TfmY.Mask), 
        ToFloatTensor(tfm_y=TfmY.Mask)]

In [ ]:
data = CamvidData(tfms=tfms)

In [ ]:
data.train.show(0)

### Biwii

In [ ]:
import pickle

In [ ]:
# export
class PointsGetter(ItemGetter):
    "An `ItemGetter` for points."
    default_tfm = TfmY.Point
    def __init__(self, procs=None, do_scale=True, y_first=False): 
        super().__init__(procs)
        self.do_scale,self.y_first = do_scale,y_first
    
    def get(self, o):
        "Inner representation of point is scaled from -1 to 1 and y first."
        if not isinstance(o, torch.Tensor): o = tensor(o)
        o = o.view(-1, 2).float()
        if not self.y_first: o = o.flip(1)
        if self.do_scale and hasattr(self, '_x') and self._x is not None: 
            sz = tensor(list(self._x.size)).float()
            o = o * 2/sz - 1
        return o
    
    def raw(self, o):
        "Put y second and unscale."
        o = o.flip(1)
        if hasattr(self, '_x') and self._x is not None: 
            sz = tensor([self._x.shape[1:]]).float()
            o = (o + 1) * sz/2
        return o
    
    def show(self, x, ax):
        params = {'s': 10, 'marker': '.', 'c': 'r'}
        ax.scatter(x[:, 1], x[:, 0], **params)

In [ ]:
items

In [ ]:
#test 
class FakeImg():
    def __init__(self, size): self.size = size
fake_img = FakeImg((200,120))

#Normal usage
get = PointsGetter()
il = ItemList([[0,0], [120,0], [0,200], [120,200], [60,100]], item_get=get)
#Without getting any context, the points aren't scaled because they don't know the picture size.
test_eq(il[1], tensor([[0., 120.]]))
with AddXContext(il, fake_img):
    test_eq(il[1], tensor([[-1., 1.]]))
    test_eq(il[4], tensor([[0., 0.]]))
    o = il[1]
#Test deproc undoes the scaling and switching when providing with right context (need to be the tensor image)
with AddXContext(il, torch.zeros(3, 120, 200)):
    test_eq(il.deproc(o), tensor([[120.,0.]]))
    
#Giving scaled points
get1 = PointsGetter(do_scale=False)
il1 = ItemList([[-1.,-1.], [1.,-1.], [-1.,1.], [1.,1.], [0.,0.]], item_get=get1)
with AddXContext(il, fake_img):
    for i in range(5): test_eq(il[i], il1[i])
with AddXContext(il1, torch.zeros(3, 120, 200)):
    for i in range(5): test_eq(il.obj(i), il1.obj(i))
        
#Giving scaled points with y_first
get2 = PointsGetter(do_scale=False, y_first=True)
il2 = ItemList([[-1.,-1.], [-1.,1.], [1.,-1.], [1.,1.], [0.,0.]], item_get=get2)
#Without getting any context, the points aren't scaled because they don't know the picture size.
with AddXContext(il, fake_img):
    for i in range(5): test_eq(il[i], il2[i])
with AddXContext(il2, torch.zeros(3, 120, 200)):
    for i in range(5): test_eq(il.obj(i), il2.obj(i))

In [ ]:
class BiwiData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = PointsGetter
    
    def get_source(self):        return untar_data(URLs.BIWI_SAMPLE)      
    def get_items(self, source): return get_image_files(source/'images')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        fn2ctr = pickle.load(open(self.source/'centers.pkl', 'rb'))
        return func_labeller(items, lambda o:fn2ctr[o.name])

In [ ]:
data = BiwiData(tfms=tfms).databunch(bs=16)

In [ ]:
data.show_batch()

### Coco

In [ ]:
#export
from fastai.vision.data import get_annotations

In [ ]:
#export
class BBoxProcessor(MultiCategoryProcessor):
    "A processor for bounding boxes."
    def create_vocab(self, items):
        super().create_vocab([c[1] for c in items])
        self.vocab.insert(0, 'background')
        self.otoi  = {v:k for k,v in enumerate(self.vocab)}

    def proc1(self, item):  return item[0],super().proc1(item[1])
    def deproc1(self, idx): return idx[0],super().deproc1(idx[1])

In [ ]:
#export 
from matplotlib import patches, patheffects

def _draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(linewidth=lw, foreground='black'), patheffects.Normal()])

def _draw_rect(ax, b, color='white', text=None, text_size=14):
    patch = ax.add_patch(patches.Rectangle(b[:2], *b[-2:], fill=False, edgecolor=color, lw=2))
    _draw_outline(patch, 4)
    if text is not None:
        patch = ax.text(*b[:2], text, verticalalignment='top', color=color, fontsize=text_size, weight='bold')
        _draw_outline(patch,1)

In [ ]:
# export
class BBoxGetter(PointsGetter):
    "An `ItemGetter` for bounding boxes."
    default_proc = BBoxProcessor
    default_tfm = TfmY.Bbox
     
    def get(self, o): return super().get(o[0]).view(-1,4),o[1]
    def raw(self, o): return super().raw(o[0].view(-1,2)).view(-1,4),o[1]
    
    def show(self, x, ax):
        bbox,label = x
        for b,l in zip(bbox, label): 
            if l != 'background': _draw_rect(ax, [b[1],b[0],b[3]-b[1],b[2]-b[0]], text=l)

In [ ]:
#export 
def bb_pad_collate(samples, pad_idx=0):
    "Collate function for bounding boxes targets."
    max_len = max([len(s[1][1]) for s in samples])
    bboxes = torch.zeros(len(samples), max_len, 4)
    labels = torch.zeros(len(samples), max_len).long() + pad_idx
    imgs = []
    for i,s in enumerate(samples):
        imgs.append(s[0][None])
        bbs, lbls = s[1]
        if not (bbs.nelement() == 0):
            bboxes[i,-len(lbls):] = bbs
            labels[i,-len(lbls):] = tensor(lbls)
    return torch.cat(imgs,0), (bboxes,labels)

In [ ]:
class CocoData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = BBoxGetter
    
    def get_source(self):        return untar_data(URLs.COCO_TINY)      
    def get_items(self, source): return get_image_files(source/'train')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        images, lbl_bbox = get_annotations(self.source/'train.json')
        img2bbox = dict(zip(images, lbl_bbox))
        return func_labeller(items, lambda o:img2bbox[o.name])
    
    def databunch(self, bs=64, **kwargs):
        kwargs['collate_fn'] = bb_pad_collate
        return super().databunch(bs=bs, **kwargs)

In [ ]:
data = CocoData(tfms=tfms).databunch(bs=16)

In [ ]:
data.show_batch()

## Export

In [ ]:
! python notebook2script.py "200_datablock_config.ipynb"